In [ ]:
!git clone https://github.com/narame7/UOS-FootballDataAnalytics-Tutorial

In [ ]:
%cd 'UOS-FootballDataAnalytics-Tutorial/Week 15'

In [ ]:
!pip install langchain==0.3.27 streamlit langchain_huggingface langchain-core==0.3.72 langchain_google_genai langchain_community==0.3.27 langchain_ollama faiss-cpu==1.11.0.post1

In [ ]:
import os
import re
import ast
from urllib.request import urlopen

import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup as bs
from sqlalchemy import create_engine, inspect
from langchain_community.utilities import SQLDatabase
# --- 디렉토리 생성 ---
BASE_DIR = os.path.join(os.getcwd(), "data")
PLAYER_STATS_DIR = os.path.join(BASE_DIR, "player_stats")
os.makedirs(PLAYER_STATS_DIR, exist_ok=True)

In [4]:
# URL 변수
REPORTURL = "https://portal.kleague.com/common/result/resultdataPopup0057.do"
INPUT_SEQ = "meetSeq"          # 리그번호(1:K1, 2:K2)
INPUT_YEAR = "year"     # 년도(2010~)
INPUT_ROUND = "round"          # 경기번호(K1기준 1~228)

# 기타 변수 - K1, K2 경기 수 정보는 매년 변경 시 수정 필요
total_games_by_league = {"K1": 228, "K2": 182}

CONSOL_LEAGUE = "Input league number(K1, K2):  "
CONSOL_YEAR = "Input year(2010~2024):  "

DATA_FRAME = ["game_number", "competition_name", "season_name", 
              "team_name", "home_team", "player_name", 
              "jersey_number", "position", "tsg_rating"]

In [ ]:
def crawling_rating(league, year):
    game_numbers = total_games_by_league[league]
    
    for game_number in tqdm(range(1, game_numbers+1)): 
        league_query = f'{INPUT_SEQ}={league[1]}'
        year_query = f'{INPUT_YEAR}={year}'
        round_query = f'{INPUT_ROUND}={game_number}'

        url = f'{REPORTURL}?{league_query}&{year_query}&{round_query}'
        report_html = urlopen(url).read()  #
 
        report = bs(report_html, 'lxml').body   # beautifulsoup 라이브러리를 통해 html을 전부 읽어오는 작업 수행
        
        title = report.find_all('td', class_='panel-bgColor panel-th')
        competition = title[1].find_next_sibling('td').text.strip()

        teams = report.find_all('td', class_='bar_bottm_right_01', colspan="4") # 홈 팀과 원정 팀 정보를 추출

        rating = []
        for team in teams:                               # [home, away]
            team_name = team.text.split('(')[0].strip()  # 팀 이름 추출
            home_team = '홈' in team.text                # 홈 팀 여부 확인

            header_row = team.find_next('tr').find_all('td')          # team's header : 팀명, 홈/원정
            headers = [header.text.strip() for header in header_row]  # player's header : 등번호, 포지션, 선수명, 평점

            # 선수 데이터
            player_rows = team.find_next('tbody').find_all('tr')

            for player in player_rows:
                player_data = player.find_all('td')
                player_dict = {headers[i]: player_data[i].text.strip() for i in range(len(headers))}
                
                player_dict['game_number'] = game_number
                player_dict['competition_name'] = competition
                player_dict['season_name'] = year
                player_dict['team_name'] = team_name
                player_dict['home_team'] = "Home" if home_team else "Away"

                rating.append(player_dict)

        rating = pd.DataFrame(rating)
        rating = rating.rename(columns = {"배번" : "jersey_number",
                                          "포지션" : "position",
                                          "선수명" : "player_name",
                                          "평점" : "tsg_rating"})

        rating[DATA_FRAME].to_csv(os.path.join(PLAYER_STATS_DIR, f'{game_number:03d}.csv'), index=False)
        
league = input(CONSOL_LEAGUE)
year = input(CONSOL_YEAR)
crawling_rating(league, year) # time: 30 seconds per season

print(f"{league} {year} TSG rating crawling completed.")

  0%|          | 0/228 [00:00<?, ?it/s]

K1 2024 TSG rating crawling completed.


In [ ]:
tsg_ratings = [pd.read_csv(os.path.join(PLAYER_STATS_DIR, f)) for f in os.listdir(PLAYER_STATS_DIR) if f.endswith('.csv')]
tsg_rating_df = pd.concat(tsg_ratings, ignore_index=True)
tsg_rating_df

,game_number,competition_name,season_name,team_name,home_team,player_name,jersey_number,position,tsg_rating
0,219,하나은행 K리그1 2024,2024,포항,Home,윤평국,1,GK,6.6
1,219,하나은행 K리그1 2024,2024,포항,Home,완델손,77,DF,7.0
2,219,하나은행 K리그1 2024,2024,포항,Home,전민광,4,DF,6.4
3,219,하나은행 K리그1 2024,2024,포항,Home,아스프로,5,DF,6.4
4,219,하나은행 K리그1 2024,2024,포항,Home,어정원,2,DF,6.4
...,...,...,...,...,...,...,...,...,...
9115,173,하나은행 K리그1 2024,2024,광주,Away,이희균,10,대기,6.8
9116,173,하나은행 K리그1 2024,2024,광주,Away,정지용,17,대기,6.6
9117,173,하나은행 K리그1 2024,2024,광주,Away,가브리엘,11,대기,6.8
9118,173,하나은행 K리그1 2024,2024,광주,Away,김한길,47,대기,0.0


In [6]:
sqlite_db_path = 'player_stats_db'
engine = create_engine(f'sqlite:///{sqlite_db_path}', echo=False)
tsg_rating_df.to_sql('player_stats', con=engine, if_exists='replace', index=False)
print(f"Player stats data saved to SQLite database '{sqlite_db_path}'.")

Player stats data saved to SQLite database 'player_stats_db'.


In [7]:
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")
print(db.table_info) # table schema 


CREATE TABLE player_stats (
	game_number BIGINT, 
	competition_name TEXT, 
	season_name BIGINT, 
	team_name TEXT, 
	home_team TEXT, 
	player_name TEXT, 
	jersey_number BIGINT, 
	position TEXT, 
	tsg_rating FLOAT
)

/*
3 rows from player_stats table:
game_number	competition_name	season_name	team_name	home_team	player_name	jersey_number	position	tsg_rating
219	하나은행 K리그1 2024	2024	포항	Home	윤평국	1	GK	6.6
219	하나은행 K리그1 2024	2024	포항	Home	완델손	77	DF	7.0
219	하나은행 K리그1 2024	2024	포항	Home	전민광	4	DF	6.4
*/


In [8]:
inspector = inspect(db._engine)
tables = inspector.get_table_names()
tables

['player_stats']

In [9]:
with db._engine.connect() as conn:
    for table in tables:
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        print(f"Loaded table '{table}' with shape {df.shape}")

df

Loaded table 'player_stats' with shape (9120, 9)


,game_number,competition_name,season_name,team_name,home_team,player_name,jersey_number,position,tsg_rating
0,219,하나은행 K리그1 2024,2024,포항,Home,윤평국,1,GK,6.6
1,219,하나은행 K리그1 2024,2024,포항,Home,완델손,77,DF,7.0
2,219,하나은행 K리그1 2024,2024,포항,Home,전민광,4,DF,6.4
3,219,하나은행 K리그1 2024,2024,포항,Home,아스프로,5,DF,6.4
4,219,하나은행 K리그1 2024,2024,포항,Home,어정원,2,DF,6.4
...,...,...,...,...,...,...,...,...,...
9115,173,하나은행 K리그1 2024,2024,광주,Away,이희균,10,대기,6.8
9116,173,하나은행 K리그1 2024,2024,광주,Away,정지용,17,대기,6.6
9117,173,하나은행 K리그1 2024,2024,광주,Away,가브리엘,11,대기,6.8
9118,173,하나은행 K리그1 2024,2024,광주,Away,김한길,47,대기,0.0


In [10]:
SQL_AGENT_PROMPT = """
    # Role
    SQLite Expert. Convert questions to SQL queries for `player_stats`.

    # Schema
    Table: `player_stats`
    Columns:
    - game_number (int): Match ID
    - competition_name (text): e.g, '하나은행 K리그1 2024'
    - season_name (int): e.g., 2024
    - team_name (text): Korean name (e.g., '포항')
    - home_team (text): 'Home' or 'Away'
    - player_name (text): Korean name
    - jersey_number (int)
    - position (text): 'GK', 'DF', 'MF', 'FW', '대기'
    - tsg_rating (real): 0.0 to 10.0

    # Critical Rules
    1. Zero Rating: `tsg_rating` 0.0 means "No Rating". Must exclude 0.0 when calculating averages (Add `WHERE tsg_rating > 0`).
    2. Text Matching: Use exact Korean strings for `team_name` and `player_name`.

    # Example
    User: "2024 포항 선수들 평균 평점"
    SQL: SELECT AVG(tsg_rating) FROM player_stats WHERE season_name = 2024 AND team_name = '포항' AND tsg_rating > 0;
"""

In [11]:
def run_query_save_results(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return res

player_names = run_query_save_results(db, "SELECT DISTINCT player_name FROM player_stats;")
team_names = run_query_save_results(db, "SELECT DISTINCT team_name FROM player_stats;")

print(f"Team names: {team_names}")
print(f"Player names: {player_names[:10]}")

Team names: ['포항', '김천', '서울', '광주', '대전', '전북', '강원', '제주', '대구', '울산', '인천', '수원FC']
Player names: ['윤평국', '완델손', '전민광', '아스프로', '어정원', '조성준', '오베르단', '김종우', '정재희', '조르지']


In [ ]:
# !curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%         1.6%  19.6%#########                        70.2%#####################                      73.5% 73.7%                   75.6% 90.3%
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# # Ollama 서버 백그라운드 실행 (포트 11434 사용)
# import subprocess
# import time

# # 로그 파일 생성
# with open("ollama.log", "w") as log_file:
#     process = subprocess.Popen(["ollama", "serve"], stdout=log_file, stderr=log_file)

# print("Ollama 서버 시작 중...")
# time.sleep(10)  # 서버가 켜질 때까지 대기

Ollama 서버 시작 중...


In [ ]:
# !ollama pull gemma2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
Error: pull model manifest: Get "https://registry.ollama.ai/v2/library/gemma2/manifests/2b": read tcp 172.17.0.11:39830->104.21.75.227:443: read: connection reset by peer


In [ ]:
# !ollama pull gemma2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
Error: pull model manifest: Get "https://registry.ollama.ai/v2/library/gemma2/manifests/latest": read tcp 172.17.0.11:43126->172.67.182.229:443: read: connection reset by peer


In [7]:
!streamlit run app.py --server.port 8501 -- --chroma_db_path ./data/namuwiki_db \
--sqlite_db_path player_stats_db \
--embedding-model "jhgan/ko-sroberta-multitask" \
--llm-model "gpt-4o-mini"

2025-12-10 23:40:44.387 Port 8501 is already in use


In [ ]:
!streamlit run app.py --server.port 8501 -- --chroma_db_path ./data/namuwiki_db \
--sqlite_db_path player_stats_db \
--embedding-model "jhgan/ko-sroberta-multitask" \
--llm-model "gemini-flash-latest"

2025-12-11 01:45:50.227 Port 8501 is already in use
